# How to use Huginn.jl

In [1]:
import Pkg
Pkg.activate(dirname(Base.current_project()))

  Activating project at `~/Julia/ODINN_notebooks`


We import Huginn.jl

In [2]:
using Huginn
using Plots

Initializing Python libraries...


Initializing Python libraries...


Initializing Python libraries...


Initializing Python libraries...


We need to specify a working directory where all data generated will be stored, similarly than it's done with OGGM.

In [3]:
working_dir = joinpath(dirname(Base.current_project()), "data")
if !ispath(working_dir)
    mkdir("data")
end

## Creating model parameters

The first thing we need to do in order to use ODINN.jl and Huginn.jl (its ice flow modelling package) is to generate some model parameters. There are multiple types of model parameters:
- `Parameters`: this parent object holds all the types of model parameters.
- `SolverParameters`: specifies all the configurations needed for the numerical solvers.
- `OGGMparametes`: contains the settings related to the use of OGGM within ODINN.jl.
- `SimulationParameters`: configures anything related to running simulations with ODINN.jl.
- `PhysicalParameters`: settings related to the physics used in the simulations. 

We can directly create a `Parameters` object, and specify any details we want for our simulation. Fields that are not specified will be filled automatically with default values.

In [4]:
params = Parameters(OGGM = OGGMparameters(working_dir=working_dir,
                                              multiprocessing=true,
                                              workers=2,
                                              ice_thickness_source = "Farinotti19"),
                        simulation = SimulationParameters(use_MB=true,
                                                          tspan=(2010.0, 2015.0),
                                                          working_dir = working_dir,
                                                          multiprocessing=true,
                                                          workers=2),
                        solver = SolverParameters(reltol=1e-8)
                        ) 

2023-09-26 11:53:06: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-09-26 11:53:06: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-09-26 11:53:06: oggm.cfg: Multiprocessing: using all available processors (N=4)
2023-09-26 11:53:06: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2023-09-26 11:53:06: oggm.cfg: PARAMS['dl_verify'] changed from `True` to `False`.
2023-09-26 11:53:06: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2023-09-26 11:53:06: oggm.cfg: Multiprocessing switched ON after user settings.
2023-09-26 11:53:06: oggm.cfg: Multiprocessing: using the requested number of processors (N=2)


Sleipnir.Parameters{PhysicalParameters{Float64}, SimulationParameters{Float64}, Nothing, SolverParameters{Float64, Int64}, Nothing, OGGMparameters}(PhysicalParameters{Float64}(900.0, 9.81, 0.001, 1.0, 8.0e-17, 8.5e-20, 1.0, -25.0, 5.0e-18), SimulationParameters{Float64}(true, true, true, true, false, Float64, Int64, (2010.0, 2015.0), 0.08333333333333333, true, 2, "/home/jovyan/Julia/ODINN_notebooks/data"), OGGMparameters("/home/jovyan/Julia/ODINN_notebooks/data", PyCall.PyDict{PyCall.PyAny, PyCall.PyAny, true}("dl_cache_dir" => "/home/jovyan/OGGM/download_cache", "tmp_dir" => "/home/jovyan/OGGM/tmp", "rgi_dir" => "/home/jovyan/OGGM/rgi", "test_dir" => "/home/jovyan/OGGM/tests", "working_dir" => "/home/jovyan/Julia/ODINN_notebooks/data", "dem_file" => "", "climate_file" => ""), PyCall.PyDict{PyCall.PyAny, PyCall.PyAny, true}("has_internet" => true, "dl_cache_readonly" => false, "use_multiprocessing" => true, "use_mp_spawn" => false, "mp_processes" => 2, "lru_maxsize" => 100, "continue_o

## Choosing a model

The next step is to specify which model we want to use for the simulations. Right now options are a little bit limited, but the flexible architecture of ODINN.jl (thanks to Sleipnir.jl), means that new ice flow, mass balance models and machine learning models can be easily added in a seamless manner. For now, we will choose the default options:

In [5]:
model = Model(iceflow = SIA2Dmodel(params), mass_balance = TImodel1(params))

Sleipnir.Model{SIA2Dmodel{Float64, Int64}, TImodel1{Float64}, Nothing}(SIA2Dmodel{Float64, Int64}(nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing), TImodel1{Float64}(0.005, 0.001), nothing)

## Initializing glaciers

The next step, is to choose which glaciers we want to work with, and initialize them. Intitializing them means downloading all the necessary topographical and climate data, which is done thanks to OGGM. Data is stored in Glacier Directories (`gdirs`), which are saved on disk as separate folders. Sleipnir.jl has a Julia interface to OGGM, which is connected to both Huginn.jl and ODINN.jl. This enables us to retrieve all sorts of data for any glacier on Earth.

First we specify the RGI IDs of the glaciers we want to simulate. All RGI IDs can be found [here](https://www.glims.org/maps/glims).

In [6]:
rgi_ids = ["RGI60-11.03638", "RGI60-11.01450", "RGI60-08.00213", "RGI60-04.04351", "RGI60-01.02170"]

5-element Vector{String}:
 "RGI60-11.03638"
 "RGI60-11.01450"
 "RGI60-08.00213"
 "RGI60-04.04351"
 "RGI60-01.02170"

Then, we initialize these glaciers in the following way.

In [7]:
glaciers = initialize_glaciers(rgi_ids, params)

┌ Info: Filtering out these glaciers from RGI ID list: String[]
└ @ Sleipnir /home/jovyan/.julia/dev/Sleipnir/src/glaciers/glacier/glacier2D_utils.jl:350
2023-09-26 11:53:19: oggm.workflow: Execute entity tasks [GlacierDirectory] on 5 glaciers
2023-09-26 11:53:20: oggm.utils: Applying global task compile_task_log on 5 glaciers


Getting raw climate data for: RGI60-11.03638


Getting raw climate data for: RGI60-11.01450


2023-09-26 11:53:22: MBsandbox.mbmod_daily_oneflowline: (RGI60-11.03638) process_w5e5_data


Getting raw climate data for: RGI60-08.00213


...

Getting raw climate data for: RGI60-01.02170


2023-09-26 11:53:39: MBsandbox.mbmod_daily_oneflowline: (RGI60-04.04351) process_w5e5_data


2023-09-26 11:53:44: MBsandbox.mbmod_daily_oneflowline: (RGI60-01.02170) process_w5e5_data


5-element Vector{Glacier2D}:
 Glacier2D{Float64, Int64}("RGI60-11.03638", PyObject <oggm.GlacierDirectory>
  RGI id: RGI60-11.03638
  Region: 11: Central Europe
  Subregion: 11-01: Alps                            
  Name: Fr4N01235A08 Dargentiere
  Glacier type: Glacier
  Terminus type: Land-terminating
  Status: Glacier or ice cap
  Area: 13.795 km2
  Lon, Lat: (6.985, 45.951)
  Grid (nx, ny): (138, 129)
  Grid (dx, dy): (62.0, -62.0)
, Climate2D{Float64}(PyObject <xarray.Dataset>
Dimensions:   (time: 1827)
Coordinates:
  * time      (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2015-01-01
Data variables:
    prcp      (time) float32 ...
    temp      (time) float64 -6.147 -12.03 -17.27 ... -18.98 -16.58 -11.98
    gradient  (time) float32 -0.005414 -0.005414 ... -0.005462 -0.005525
Attributes:
    ref_hgt:         1940.0
    ref_pix_lon:     6.75
    ref_pix_lat:     45.75
    ref_pix_dis:     28823.67358258575
    climate_source:  temp: W5E5_daily, prcp: W5E5_daily, lapse rate: ER

/home/jovyan/.local/lib/python3.9/site-packages/oggm/utils/_workflow.py:2828: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return np.round(float(_area), decimals=3)
/home/jovyan/.local/lib/python3.9/site-packages/oggm/utils/_workflow.py:2828: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return np.round(float(_area), decimals=3)
/home/jovyan/.local/lib/python3.9/site-packages/oggm/utils/_workflow.py:2828: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return np.round(float(_area), decimals=3)
/home/jovyan/.local/lib/python3.9/site-packages/oggm/utils/_workflow.py:2828: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.i

## Create a simulation

In the modelling framework of ODINN.jl, we can have different types of simulations. For now, the main ones are:
- `Prediction`: A forward run of a differential equation describing ice flow, with or without mass balance.
- `Inversion`: A classic inversion of one or more parameters. Can be steady state or transient, performed by computing the gradients of the solution of the differential equation w.r.t. the parameters of interest.
- `FunctionalInversion`: The inversion of a function, captured by a neural network embedded in a differential equation (i.e. a Universal Differential Equation).

For now, we will stick to the simplest and most tested case, a `Prediction`. A simulation is composed by a model, a list of glaciers and a set of parameters.

In [8]:
prediction = Prediction(model, glaciers, params)

Prediction(Sleipnir.Model{SIA2Dmodel{Float64, Int64}, TImodel1{Float64}, Nothing}(SIA2Dmodel{Float64, Int64}(nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing), TImodel1{Float64}(0.005, 0.001), nothing), AbstractGlacier[Glacier2D{Float64, Int64}("RGI60-11.03638", PyObject <oggm.GlacierDirectory>
  RGI id: RGI60-11.03638
  Region: 11: Central Europe
  Subregion: 11-01: Alps                            
  Name: Fr4N01235A08 Dargentiere
  Glacier type: Glacier
  Terminus type: Land-terminating
  Status: Glacier or ice cap
  Area: 13.795 km2
  Lon, Lat: (6.985, 45.951)
  Grid (nx, ny): (138, 129)
  Grid (dx, dy): (62.0, -62.0)
, Climate2D{Float64}(PyObject <xarray.Dataset>
Dimensions:   (time: 1827)
Coordinates:
  * time      (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2015-01-01
Data vari

## Running a simulation

Once we have everything ready, we can easily run the chosen simulation in the following manner.

In [9]:
run!(prediction)

Running forward in-place PDE ice flow model...



Processing glacier: RGI60-11.03638


Processing glacier: RGI60-11.01450


Processing glacier: RGI60-08.00213


Progress:  40%|████████████████▍                        |  ETA: 0:00:27

Processing glacier: RGI60-04.04351


Processing glacier: RGI60-01.02170


Progress:  80%|████████████████████████████████▊        |  ETA: 0:00:05

Progress: 100%|█████████████████████████████████████████| Time: 0:00:22


Once the simulation has been run, we can access all the results in `prediction.results`. 

In [10]:
prediction.results

5-element Vector{Results}:
 Results{Float64}("RGI60-11.03638", [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], [1444.0 1462.0 … 3049.0 3041.0; 1415.0 1428.0 … 3034.0 3027.0; … ; 2009.0 2011.0 … 3202.0 3211.0; 2016.0 2016.0 … 3151.0 3171.0], [1444.0 1462.0 … 3049.0 3041.0; 1415.0 1428.0 … 3034.0 3027.0; … ; 2009.0 2011.0 … 3202.0 3211.0; 2016.0 2016.0 … 3151.0 3171.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; -0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0], [-0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0])
 Results{Float64}("RGI60-11.01450", [[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], [846.0 1146.0 … 2498.0 2588.0; 854.0 1126.0 … 2